# Samurai Gunn map tile classification

### First define the window capture function
On it's own, it seems to barely have any performance impact, outputting up to 800-900 fps when capturing the windowed version of Samurai Gunn, a 320x240 px window. 

In [1]:
import cv2 as cv
import numpy as np
from time import time
import win32gui
import win32ui
import win32con

def list_all_open_windows():
    def winEnumHandler( hwnd, ctx ):
        if win32gui.IsWindowVisible( hwnd ):
            print (hex(hwnd), win32gui.GetWindowText( hwnd ))

    win32gui.EnumWindows(winEnumHandler, None)

#list_all_open_windows()

def window_capture():
    
    hwnd = win32gui.FindWindow(None, 'Samurai Gunn')
    #hwnd = None
    
    # Getting the window's size and accounting for window screenshot borders
    #window_rect = win32gui.GetWindowRect(hwnd)
    titlebar_px = 38
    border_px = 9

    # For samurai Gunn, the non-fullscreen dimensions should be:
    w = 320
    h = 240
    
    crop_x = border_px
    crop_y = titlebar_px
    
    wDC = win32gui.GetWindowDC(hwnd)
    dcObj = win32ui.CreateDCFromHandle(wDC)
    cDC = dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (w, h) , dcObj, (crop_x, crop_y), win32con.SRCCOPY)
    
    
    # saving screenshot to file
    # To save screenshot to file, uncomment this 
    #bmpfilenamename = "sample" + str(img_counter) + ".jpg" #set this
    #dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)
    
    # Converting to format useful for opencv
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    img = np.frombuffer(signedIntsArray, dtype='uint8')
    img.shape = (h, w, 4)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())
    
    # Dropping alpha channel may be useful for some applications, like cv.matchTemplate()
    # which may throw an error otherwise
    #img = img[...,:3]   # this drops alpha channel 
    
    return img

In [29]:
'''
tiles are (16 x 16)px
 x, y  : coords of the upper left corner of the tile
 sliced: int, either 0 or 1, letting us know if it's a full tile or a half tile at the 
         upper/lower edges of the map. 
         0 means it's a full tile
         1 means it's a half tile
img: 
         
 returns a single int that will classify the tile
'''
row_step = 2
col_step = 2
def classify_tile(x, y, img, sliced):
    px_count = 0
    total = np.zeros((1, img.shape[2]), dtype='uint32')
    #print('total type ' + str(type(total)))
    
    corrected_height_range = int(16/(1+sliced))
    for i in range(0, corrected_height_range, row_step):
        for j in range(0, 16, col_step):
            total += img[x+i][y+j] # [r, g, b, a]
            px_count += 1           
    #print('total: ' + str(total))
    #print('px count: ' + str((1+sliced)/(px_count)))
    result = total * ((1+sliced)/(px_count))
    #print('resulting color ' + str(result))
    #result = result.astype('uint8')
    
    return result
    

In [33]:
'''
The tiles in every map can be aligned with a 20 x 15 grid of (16 x 16)px cells
img: input image
offset: in case the map tiles do not align perfectly with the grid, an offset of 8px down does 
        the trick. (Maps with different offsets have yet to be found, 
        this should allow for different offsets in the X direction)
'''
def simplify(img, offset):
    sliced = 0
    simple_img = np.zeros((15, 20, img.shape[2]))
    for x in range(15):
        #print('x = ' + str(x))
        if offset == 8 and (x == 0 or x == 19):
            sliced = 1
        else:
            sliced = 0
        for y in range(20):
            # pass the coordinates of each tile with corrections to accomodate for the grid offset
            simple_img[x, y] = classify_tile(x*16 - (offset*(1-sliced)), 
                                             y*16 - (offset*(1-sliced)), 
                                             img, sliced)
            #print(simple_img[x, y])
    simple_img = simple_img.astype('uint8')
    return simple_img

In [34]:
counter = 1
total = 0

# offset will depend on the map
# for Ice cube, offset is 0
offset = 0

while(True):
    prev_time = time()
    
    screenshot = window_capture()
    #screenshot = cv.imread('sample0.jpg')
    #print(screenshot.shape)
    img = simplify(screenshot, offset)
    dim = (img.shape[1] * 16, img.shape[0] * 16)
    resized = cv.resize(img, dim, interpolation = cv.INTER_AREA) #  interpolation = cv.INTER_AREA
    #resized = resized.astype('uint8')
    cv.imshow('Simplified', resized)
    cv.imshow('Screenshot', screenshot)
    #print('FPS {}'.format(1 / (time() - prev_time)))
     
    total += 1 / (time() - prev_time)
    print('Avg FPS {}'.format(total/counter))
    counter += 1
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break

Avg FPS 9.682075147217356
Avg FPS 16.235258834760298
Avg FPS 19.179109256541725
Avg FPS 20.645272119980795
Avg FPS 21.657808359905395
Avg FPS 22.12104101927297
Avg FPS 22.684118793635843
Avg FPS 23.21207642593922
Avg FPS 23.724329981652787
Avg FPS 23.99804539427322
Avg FPS 24.158731770504094
Avg FPS 24.39031050772634
Avg FPS 24.542121243679183
Avg FPS 24.57799535909719
Avg FPS 24.700217633517443
Avg FPS 24.684785508833794
Avg FPS 24.70714299260309
Avg FPS 24.80046307009363
Avg FPS 24.84942929670813
Avg FPS 24.94223015165268
Avg FPS 24.890889946588747
Avg FPS 24.844680110358265
Avg FPS 24.83969960966545
Avg FPS 24.94676516737142
Avg FPS 25.02052398874795
Avg FPS 24.997968158402706
Avg FPS 25.068158781778486
Avg FPS 25.142593960118095
Avg FPS 25.184706239945516
Avg FPS 25.22398341641058
Avg FPS 25.307903184649675
Avg FPS 25.36186124119688
Avg FPS 25.416347390363605
Avg FPS 25.374390002711273
Avg FPS 25.403366417658685
Avg FPS 25.27172974848962
Avg FPS 25.320477529479483
Avg FPS 25.370651

Avg FPS 25.552628954942318
Avg FPS 25.555209069729276
Avg FPS 25.55522254201116
Avg FPS 25.56144921407546
Avg FPS 25.543334587893767
Avg FPS 25.548265295671303
Avg FPS 25.548299944464546
Avg FPS 25.546967075375317
Avg FPS 25.553998156400475
Avg FPS 25.55264365360695
Avg FPS 25.555320710930054
Avg FPS 25.548341919518776
Avg FPS 25.556123663295352
Avg FPS 25.5601415617374
Avg FPS 25.564950738384788
Avg FPS 25.563433786901065
Avg FPS 25.56812557544186
Avg FPS 25.564752228580097
Avg FPS 25.569819044353558
Avg FPS 25.570182612706116
Avg FPS 25.570548730712108
Avg FPS 25.562252854419985
Avg FPS 25.564790020938112
Avg FPS 25.561481514867335
Avg FPS 25.554891438811214
Avg FPS 25.550037763856682
Avg FPS 25.551148103437445
Avg FPS 25.549816727013635
Avg FPS 25.552278261302057
Avg FPS 25.5473016785184
Avg FPS 25.539227961040655
Avg FPS 25.53966845538905
Avg FPS 25.540161778288937
Avg FPS 25.494338904399726
Avg FPS 25.476380151677308
Avg FPS 25.483226084231216
Avg FPS 25.449970954762673
Avg FPS 25

Avg FPS 25.837284992277414
Avg FPS 25.84012084783115
Avg FPS 25.844162847202337
Avg FPS 25.838430806697616
Avg FPS 25.84175947023719
Avg FPS 25.84532016446051
Avg FPS 25.84827086940643
Avg FPS 25.85071123914059
Avg FPS 25.85269582130302
Avg FPS 25.85568151997617
Avg FPS 25.858842491121518
Avg FPS 25.861357575876184
Avg FPS 25.86450368321298
Avg FPS 25.8680949724645
Avg FPS 25.871025403828405
Avg FPS 25.862476627308173
Avg FPS 25.863261111479996
Avg FPS 25.85621883638992
Avg FPS 25.85933997149727
Avg FPS 25.86209085915433
Avg FPS 25.859515839690236
Avg FPS 25.862579839723754
Avg FPS 25.866068132903063
Avg FPS 25.868446898707624
Avg FPS 25.871584361567233
Avg FPS 25.875948429556523
Avg FPS 25.877836627992725
Avg FPS 25.88099465151042
Avg FPS 25.884314639749547
Avg FPS 25.88661613544064
Avg FPS 25.888575273160427
Avg FPS 25.890425598017703
Avg FPS 25.889159272037375
Avg FPS 25.892117773960557
Avg FPS 25.896088798387186
Avg FPS 25.89683623373512
Avg FPS 25.895743050329568
Avg FPS 25.895453

Avg FPS 25.727241283464135
Avg FPS 25.72878428304981
Avg FPS 25.7295427046403
Avg FPS 25.73157896011565
Avg FPS 25.731172349709006
Avg FPS 25.73260479073766
Avg FPS 25.734868027111986
Avg FPS 25.730092077165974
Avg FPS 25.731415491826063
Avg FPS 25.733051294524323
Avg FPS 25.734320852658232
Avg FPS 25.73577892715421
Avg FPS 25.730165309250307
Avg FPS 25.732384552937358
Avg FPS 25.734965686604138
Avg FPS 25.735987433257915
Avg FPS 25.736970951195342
Avg FPS 25.73782159649767
Avg FPS 25.739312596049878
Avg FPS 25.73996929977813
Avg FPS 25.742431263271715
Avg FPS 25.74247044173068
Avg FPS 25.741231047722543
Avg FPS 25.74369933021878
Avg FPS 25.7331353928056
Avg FPS 25.73241259505399
Avg FPS 25.734241208584834
Avg FPS 25.73656052485324
Avg FPS 25.73888881317552
Avg FPS 25.737743604336753
Avg FPS 25.739225937615448
Avg FPS 25.73918995304304
Avg FPS 25.741425602604576
Avg FPS 25.742495695795345
Avg FPS 25.743899239873286
Avg FPS 25.746879554477694
Avg FPS 25.746173222811375
Avg FPS 25.749171

Avg FPS 25.85200791753874
Avg FPS 25.85299677760637
Avg FPS 25.854288417991327
Avg FPS 25.853811430237975
Avg FPS 25.854165913144076
Avg FPS 25.855805175979828
Avg FPS 25.856527984096754
Avg FPS 25.856404696830385
Avg FPS 25.857399350253413
Avg FPS 25.857340511727312
Avg FPS 25.85896324708812
Avg FPS 25.85938706093141
Avg FPS 25.860618821957466
Avg FPS 25.861672052824343
Avg FPS 25.862519479453
Avg FPS 25.858218909914815
Avg FPS 25.85864004097072
Avg FPS 25.859061162289805
Avg FPS 25.86093739764233
Avg FPS 25.862458624214895
Avg FPS 25.863905067207046
Avg FPS 25.860298789632726
Avg FPS 25.861601845657106
Avg FPS 25.86186000128628
Avg FPS 25.862297949679057
Avg FPS 25.86268151430377
Avg FPS 25.863136284688046
Avg FPS 25.86409066228303
Avg FPS 25.865571916460194
Avg FPS 25.866527002610383
Avg FPS 25.86818271040242
Avg FPS 25.869818497162697
Avg FPS 25.869690028842914
Avg FPS 25.871258421925695
Avg FPS 25.871114430567204
Avg FPS 25.87149937237283
Avg FPS 25.872866252039888
Avg FPS 25.8743

...

Avg FPS 26.032204545883207
Avg FPS 26.03135116663392
Avg FPS 26.03149445566814
Avg FPS 26.031448128393773
Avg FPS 26.031537150889733
Avg FPS 26.031584373723668
Avg FPS 26.031724009031144
Avg FPS 26.031875112141297
Avg FPS 26.03158400877167
Avg FPS 26.031543987734533
Avg FPS 26.030636451862595
Avg FPS 26.03089553474461
Avg FPS 26.031091924160428
Avg FPS 26.0304020577572
Avg FPS 26.029545821996557
Avg FPS 26.02979029741866
Avg FPS 26.029920872987926
Avg FPS 26.030223244733087
Avg FPS 26.03026791641351
Avg FPS 26.029466785054066
Avg FPS 26.029609934069207
Avg FPS 26.029720893680125
Avg FPS 26.029309068278575
Avg FPS 26.029552842809
Avg FPS 26.02959754618403
Avg FPS 26.029886300061715
Avg FPS 26.030232859749933
Avg FPS 26.03048578559087
Avg FPS 26.030756063068207
Avg FPS 26.03099967093411
Avg FPS 26.030309424985276
Avg FPS 26.029682391840684
Avg FPS 26.029833686192237
Avg FPS 26.03007721288484
Avg FPS 26.03029908472992
Avg FPS 26.030546048539428
Avg FPS 26.03058665218372
Avg FPS 26.0299714

Avg FPS 26.027570695742746
Avg FPS 26.02781337416971
Avg FPS 26.028048222190126
Avg FPS 26.02756323889984
Avg FPS 26.027764213312285
Avg FPS 26.027810218512286
Avg FPS 26.02739582455898
Avg FPS 26.027438979653102
Avg FPS 26.027583461972647
Avg FPS 26.026919157912094
Avg FPS 26.027032989927484
Avg FPS 26.027267832941565
Avg FPS 26.02749949763275
Avg FPS 26.027545532279756
Avg FPS 26.02778048053676
Avg FPS 26.027834350453166
Avg FPS 26.028019466560938
Avg FPS 26.027537523705938
Avg FPS 26.02767697368108
Avg FPS 26.026910538278326
Avg FPS 26.027139067258823
Avg FPS 26.027375451295363
Avg FPS 26.026712672597586
Avg FPS 26.026943819716983
Avg FPS 26.02715735558688
Avg FPS 26.02737356508631
Avg FPS 26.027604799143855
Avg FPS 26.026889410715906
Avg FPS 26.02712399670255
Avg FPS 26.027335822389528
Avg FPS 26.027520091729173
Avg FPS 26.02775054173773
Avg FPS 26.027889826082856
Avg FPS 26.02809390662774
Avg FPS 26.028328180852693
Avg FPS 26.02820457220856
Avg FPS 26.02807832359087
Avg FPS 26.028

Avg FPS 26.024421441102206
Avg FPS 26.024602997245815
Avg FPS 26.02473619962891
Avg FPS 26.024097593842153
Avg FPS 26.024300333020125
Avg FPS 26.024523305566156
Avg FPS 26.024564799155446
Avg FPS 26.02377602179304
Avg FPS 26.02408462662847
Avg FPS 26.02421452772894
Avg FPS 26.02454025285036
Avg FPS 26.024861659981045
Avg FPS 26.025191860039378
Avg FPS 26.024652172589917
Avg FPS 26.02398925946
Avg FPS 26.0239557291195
Avg FPS 26.023322948331078
Avg FPS 26.02305438873917
Avg FPS 26.023375942990768
Avg FPS 26.022851162698775
Avg FPS 26.021698095779303
Avg FPS 26.022027737039544
Avg FPS 26.021625466421163
Avg FPS 26.021848387556837
Avg FPS 26.021109178132424
Avg FPS 26.021309777183447
Avg FPS 26.020907824767324
Avg FPS 26.021134012656624
Avg FPS 26.020346204220505
Avg FPS 26.020562425740525
Avg FPS 26.02078544506408
Avg FPS 26.020323673956007
Avg FPS 26.01949123234073
Avg FPS 26.019714301329028
Avg FPS 26.019925524825588
Avg FPS 26.02004365468143
Avg FPS 26.02013512994479
Avg FPS 26.020360

Avg FPS 26.019044970168135
Avg FPS 26.019235163735587
Avg FPS 26.019522830301
Avg FPS 26.018925864704762
Avg FPS 26.018969415478917
Avg FPS 26.019248797102815
Avg FPS 26.018406657726498
Avg FPS 26.017989734559237
Avg FPS 26.01811625639235
Avg FPS 26.017405799244376
Avg FPS 26.017554554488925
Avg FPS 26.017719846578185
Avg FPS 26.017839391710282
Avg FPS 26.01805472810273
Avg FPS 26.018127005707857
Avg FPS 26.017414906437796
Avg FPS 26.01758411251566
Avg FPS 26.01763495271786
Avg FPS 26.01786164081084
Avg FPS 26.018077172533957
Avg FPS 26.01820571243611
Avg FPS 26.018421481748486
Avg FPS 26.018636671857887
Avg FPS 26.018879432955757
Avg FPS 26.01869730714982
Avg FPS 26.018992898925227
Avg FPS 26.01922743912143
Avg FPS 26.019361329812263
Avg FPS 26.019579861759098
Avg FPS 26.01970359099842
Avg FPS 26.019954868553043
Avg FPS 26.020153645389783
Avg FPS 26.020462955227707
Avg FPS 26.020734512604008
Avg FPS 26.021042797281748
Avg FPS 26.021351240788608
Avg FPS 26.02155674852058
Avg FPS 26.021

Avg FPS 26.04234739795663
Avg FPS 26.041972754935067
Avg FPS 26.04209438137388
Avg FPS 26.04171983945672
Avg FPS 26.04123138890884
Avg FPS 26.041354610599615
Avg FPS 26.04146705407771
Avg FPS 26.0416724923931
Avg FPS 26.04189812068241
Avg FPS 26.041996692378248
Avg FPS 26.042200980165795
Avg FPS 26.042410746332507
Avg FPS 26.042514529635586
Avg FPS 26.04263581924222
Avg FPS 26.04209525617849
Avg FPS 26.042399802163462
Avg FPS 26.042600831356793
Avg FPS 26.042773232242464
Avg FPS 26.042894382226123
Avg FPS 26.04301989464028
Avg FPS 26.04320100034884
Avg FPS 26.043419665851868
Avg FPS 26.043627305859946
Avg FPS 26.04383470707241
Avg FPS 26.043304313214108
Avg FPS 26.043186550165927
Avg FPS 26.043262528101952
Avg FPS 26.04322701316642
Avg FPS 26.043266357598753
Avg FPS 26.043470943747234
Avg FPS 26.043599538369815
Avg FPS 26.043745986107332
Avg FPS 26.043953117174258
Avg FPS 26.043802064075553
Avg FPS 26.044042049251463
Avg FPS 26.044333766272793
Avg FPS 26.04329623148741
Avg FPS 26.04351